# Neural Collaborative Filtering

## Matrix factorization algorithm

NCF - это нейронная модель матричной факторизации, которая объединяет Generalized Matrix Factorization (GMF) и Multi-Layer Perceptron (MLP), объединяя в себе сильные стороны линейности MF и нелинейности MLP для моделирования скрытых структур user-item.

Схема архитектуры NCF:

<img src="https://recodatasets.blob.core.windows.net/images/NCF.svg?sanitize=true">

На схеме видно, как используются скрытые вектора пользователей и айтемов и как затем объединяются выходы из слоя GMF (слева) и слоя MLP (справа).

### 1.1 Модель GMF

В ALS, матрицу оценок можно записать как:

$$\hat { r } _ { u , i } = q _ { i } ^ { T } p _ { u }$$

GMF представляет собой слой NCF как стандартный выходной слой MF. Поэтому MF может быть легко обобщена и расширена. Например, если мы позволим веса ребер выходно слоя обучаться без общего ограничения - это даст вариант MF, который позволяет варьировать важность скрытых измерений. А если мы будем использовать нелинейную функцию активации, это даст обобщение MF до нелинейной формы, которая может быть более выразительной чем линейная MF модель. GMF может быть записана как:

$$\hat { r } _ { u , i } = a _ { o u t } \left( h ^ { T } \left( q _ { i } \odot p _ { u } \right) \right)$$

где $\odot$ - поэлементное произведение векторов, ${a}_{out}$ и ${h}$ обозначают функцию активации и веса ребер выходного слоя соответственно. MF может рассматриваться как частный случай GMF. Интуитивно, если мы используем тождественную функцию для ${a}_{out}$ и выбираем единичный вектор в качестве ${h}$, то мы в точности повторяем модель MF.


### 1.2 Модель MLP

NCF использует два способа при моделировании рейтингов:

1) поэлементное произведение векторов,
2) конкатенация векторов.

Сразу после контатенации скрытых признаков пользователей и айтемов применяется стандартная модель MLP. Это дает возможность наделить модель большим уровнем гибкости и нелинейности для изучения взаимодействий между $p_{u}$ и $q_{i}$. 

Запишем модель MLP модель более строго:

Для входного слоя, используется конкатенация векторов пользователей и айтемов:

$$z _ { 1 } = \phi _ { 1 } \left( p _ { u } , q _ { i } \right) = \left[ \begin{array} { c } { p _ { u } } \\ { q _ { i } } \end{array} \right]$$

Для скрытых и выходного слоев MLP запись имеет вид:

$$
\phi _ { l } \left( z _ { l } \right) = a _ { o u t } \left( W _ { l } ^ { T } z _ { l } + b _ { l } \right) , ( l = 2,3 , \ldots , L - 1 )
$$

и:

$$
\hat { r } _ { u , i } = \sigma \left( h ^ { T } \phi \left( z _ { L - 1 } \right) \right)
$$

где ${ W }_{ l }$, ${ b }_{ l }$, и ${ a }_{ out }$ обозначают матрицу весов, вектор свободных членов и функцию активации для $l$-ого слоя, соответственно. В качестве функций активации MLP слоев, мы вольны выбирать любую: сигмоиду, гиперболический тангенс, ReLU и другие. В качестве функции активации на выходном слое используется сигмоида $\sigma(x)=\frac{1}{1+e^{-x}}$, чтобы ограничить оценки диапазоном (0,1).

### 1.3 Смешивание GMF и MLP

Чтобы обеспечить большую гибкость нашей смешанной модели мы позволяем GMF и MLP обучать независимые эмбединги и затем комбинируем две модели объединяя их последние скрытие слои. Мы взяли $\phi^{GMF}$ из GMF:

$$\phi _ { u , i } ^ { G M F } = p _ { u } ^ { G M F } \odot q _ { i } ^ { G M F }$$

и получили $\phi^{MLP}$ из MLP:

$$\phi _ { u , i } ^ { M L P } = a _ { o u t } \left( W _ { L } ^ { T } \left( a _ { o u t } \left( \ldots a _ { o u t } \left( W _ { 2 } ^ { T } \left[ \begin{array} { c } { p _ { u } ^ { M L P } } \\ { q _ { i } ^ { M L P } } \end{array} \right] + b _ { 2 } \right) \ldots \right) \right) + b _ { L }\right.$$

Наконец, мы смешали выходы из GMF и MLP:

$$\hat { r } _ { u , i } = \sigma \left( h ^ { T } \left[ \begin{array} { l } { \phi ^ { G M F } } \\ { \phi ^ { M L P } } \end{array} \right] \right)$$

Модель сочетает линейность MF и нелинейность DNN при моделировании скрытых user–item структур.

### 1.4 Целевая функция

Мы можем записать функцию правдоподобия как:

$$P \left( \mathcal { R } , \mathcal { R } ^ { - } | \mathbf { P } , \mathbf { Q } , \Theta \right) = \prod _ { ( u , i ) \in \mathcal { R } } \hat { r } _ { u , i } \prod _ { ( u , j ) \in \mathcal { R } ^{ - } } \left( 1 - \hat { r } _ { u , j } \right)$$

Где $\mathcal{R}$ обозначает множество наблюдаемых взаимодействий пользователя, а $\mathcal{ R } ^ { - }$ обозначает множество негативных наблюдений. $\mathbf{P}$ и $\mathbf{Q}$ - это скрытая матрица признаков пользователей и айтемов соответственно, $\Theta$ - параметры модели. Взяв со знаком минус логарифм от правдоподобия мы получим целевую функцию для минимизации NCF алгоритма. Что-то напоминает, не правда ли? https://en.wikipedia.org/wiki/Cross_entropy

$$L = - \sum _ { ( u , i ) \in \mathcal { R } \cup { \mathcal { R } } ^ { - } } r _ { u , i } \log \hat { r } _ { u , i } + \left( 1 - r _ { u , i } \right) \log \left( 1 - \hat { r } _ { u , i } \right)$$

## Tensorflow NCF implementation

### Downloading dataset

In [0]:
!wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.negative
!wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.rating
!wget https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.train.rating

--2020-02-05 22:38:28--  https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.negative
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2891424 (2.8M) [text/plain]
Saving to: ‘ml-1m.test.negative.1’

ml-1m.test.negative 100%[===================>]   2.76M  13.4MB/s    in 0.2s    

2020-02-05 22:38:29 (13.4 MB/s) - ‘ml-1m.test.negative.1’ saved [2891424/2891424]

--2020-02-05 22:38:30--  https://raw.githubusercontent.com/hexiangnan/neural_collaborative_filtering/master/Data/ml-1m.test.rating
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request 

### Imports

In [0]:
import math
import heapq
import scipy.sparse as sp
import multiprocessing
from six.moves import xrange

from time import time
import sys

import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
print("TF version:", tf.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.1.0
GPU is available


### Constants

In [0]:
# ml-1m dataset contains 1,000,209 anonymous ratings of approximately 3,706 movies made by 6,040 users who joined MovieLens in 2000.
# All ratings are contained in the file "ratings.dat" without header row, and are in the following format:
# UserID::MovieID::Rating::Timestamp
#
# - UserIDs range between 1 and 6040.
# - MovieIDs range between 1 and 3952.
# - Ratings are made on a 5-star scale (whole-star ratings only).

FILE_NAME = 'ml-1m'

USER_COLUMN = "user_id"
ITEM_COLUMN = "item_id"  # movies
RATING_COLUMN = "rating"

### Data loading

Данные предварительно предобработаны:

**train.rating:**
- Train file.
- Each Line is a training instance: userID itemID rating timestamp (if have)

**test.rating:**
- Test file (positive instances). 
- Each Line is a testing instance: userID itemID rating timestamp (if have)

**test.negative**
- Test file (negative instances).
- Each line corresponds to the line of test.rating, containing 99 negative samples.  
- Each line is in the format: (userID,itemID) negativeItemID1 negativeItemID2 ...

In [0]:
### Ничего интересного - чтение данных из файлов (можно пропустить).

def load_rating_file_as_list(filename):
    ratingList = []
    with open(filename+'.test.rating', "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item = int(arr[0]), int(arr[1])
            ratingList.append([user, item])
            line = f.readline()
    return ratingList

def load_negative_file(filename):
    negativeList = []
    with open(filename+'.test.negative', "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            negatives = []
            for x in arr[1: ]:
                negatives.append(int(x))
            negativeList.append(negatives)
            line = f.readline()
    return negativeList

def load_rating_file_as_matrix(filename):
    '''
    Read .rating file and Return dok matrix.
    '''
    # Get number of users and items
    num_users, num_items = 0, 0
    with open(filename+'.train.rating', "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            u, i = int(arr[0]), int(arr[1])
            num_users = max(num_users, u)
            num_items = max(num_items, i)
            line = f.readline()
    # Construct matrix
    mat = sp.dok_matrix((num_users+1, num_items+1), dtype=np.float32)
    with open(filename+'.train.rating', "r") as f:
        line = f.readline()
        while line != None and line != "":
            arr = line.split("\t")
            user, item, rating = int(arr[0]), int(arr[1]), float(arr[2])
            if (rating > 0):
                mat[user, item] = 1.0
            line = f.readline()    
    return mat

In [0]:
t1 = time()

# Loading data
train, testRatings, testNegatives = load_rating_file_as_matrix(FILE_NAME), load_rating_file_as_list(FILE_NAME), load_negative_file(FILE_NAME)
num_users, num_items = train.shape

print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d" 
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

Load data done [14.6 s]. #user=6040, #item=3706, #train=994169, #test=6040


### Metrics functions

In [0]:
### Расчет метрик: Hit_Ratio, NDCG для top-K рекомендаций

# Global variables that are shared across processes
_model = None
_testRatings = None
_testNegatives = None
_K = None

def getHitRatio(ranklist, gtItem):
    for item in ranklist:
        if item == gtItem:
            return 1
    return 0

def getNDCG(ranklist, gtItem):
    for i in xrange(len(ranklist)):
        item = ranklist[i]
        if item == gtItem:
            return math.log(2) / math.log(i+2)
    return 0

def eval_one_rating(idx):
    rating = _testRatings[idx]
    items = _testNegatives[idx]
    u = rating[0]
    gtItem = rating[1]
    items.append(gtItem)
    # Get prediction scores
    map_item_score = {}
    users = np.full(len(items), u, dtype = 'int32')
    predictions = _model.predict([users, np.array(items)], 
                                 batch_size=100, verbose=0)
    for i in xrange(len(items)):
        item = items[i]
        map_item_score[item] = predictions[i]
    items.pop()
    
    # Evaluate top rank list
    ranklist = heapq.nlargest(_K, map_item_score, key=map_item_score.get)
    hr = getHitRatio(ranklist, gtItem)
    ndcg = getNDCG(ranklist, gtItem)
    return (hr, ndcg)

def evaluate_model(model, testRatings, testNegatives, K, num_thread):

    global _model
    global _testRatings
    global _testNegatives
    global _K
    _model = model
    _testRatings = testRatings
    _testNegatives = testNegatives
    _K = K
        
    hits, ndcgs = [],[]
    if(num_thread > 1): # Multi-thread
        pool = multiprocessing.Pool(processes=num_thread)
        res = pool.map(eval_one_rating, range(len(_testRatings)))
        pool.close()
        pool.join()
        hits = [r[0] for r in res]
        ndcgs = [r[1] for r in res]
        return (hits, ndcgs)
    # Single thread
    for idx in xrange(len(_testRatings)):
        (hr,ndcg) = eval_one_rating(idx)
        hits.append(hr)
        ndcgs.append(ndcg)      
    return (hits, ndcgs)

### Model building

In [0]:
mf_regularization = 0. #regularization factor for MF embeddings

def get_model(num_users, num_items, mf_dim=10, model_layers=[16, 8], mlp_reg_layers=[0., 0.], reg_mf=0):
    assert len(model_layers) == len(mlp_reg_layers)

    # Input variables

    user_input = tf.keras.layers.Input( """ your code here """ )
    
    item_input = tf.keras.layers.Input( """ your code here """ )

    # Embedding layer

    # Initializer for embedding layers
    embedding_initializer = "glorot_uniform"

    # It turns out to be significantly more effecient to store the MF and MLP
    # embedding portions in the same table, and then slice as needed.
    embedding_user = tf.keras.layers.Embedding( """ your code here """ )

    embedding_item = tf.keras.layers.Embedding( """ your code here """ )

    # GMF part

    def mf_slice_fn(x):
       """ your code here """ 
    mf_user_latent = tf.keras.layers.Lambda( """ your code here """ )
    mf_item_latent = tf.keras.layers.Lambda( """ your code here """ )
    
    # Element-wise multiply
    mf_vector =  """ your code here """ 

    # MLP part

    def mlp_slice_fn(x):
       """ your code here """ 
    mlp_user_latent = tf.keras.layers.Lambda( """ your code here """ )
    mlp_item_latent = tf.keras.layers.Lambda( """ your code here """ )

    # Concatenation of two latent features
    mlp_vector =  """ your code here """ 

    num_layer = len(model_layers) # Number of layers in the MLP
    for layer in xrange(1, num_layer):
       """ your code here """ 

    # Concatenate GMF and MLP parts
    predict_vector =  """ your code here """   
    
    # Final prediction layer
    logits =  """ your code here """ 

    model =  """ your code here """ 

    # Print model topology.
    model.summary()

    return model

### Задания для самостоятельной работы:


1. Собрать архитектуру GMF
2. Собрать архитектуру MLP
3. Собрать архитектуру NeuMF (GMF+MLP)
4. Попробовать различные гиперпараметры: mf_regularization, lr
5. Попробовать другой оптимизатор: SGD, Adam и др.
6. 



In [0]:
model = get_model(num_users, num_items)

Model: "model_13"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_id (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_user (Embedding)      (None, 1, 18)        108720      user_id[0][0]                    
__________________________________________________________________________________________________
embedding_item (Embedding)      (None, 1, 18)        66708       item_id[0][0]                    
___________________________________________________________________________________________

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.1), loss='binary_crossentropy')

### Training

In [0]:
# обучается ~ 2 мин.
start_time = time()

# Init performance
topK = 10
evaluation_threads = 1 #multiprocessing.cpu_count()

(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()
print("Init: HR = %.4f, NDCG = %.4f" % (hr, ndcg))

train_time = time() - start_time
print("Took %.1f seconds for training." % (train_time))

best_hr, best_ndcg, best_iter = hr, ndcg, -1

Init: HR = 0.1315, NDCG = 0.0673
Took 135.5 seconds for training.


In [0]:
def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in xrange(num_negatives):
            j = np.random.randint(num_items)
            while (u, j) in train:
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

In [0]:
num_epochs = 1 #20
num_negatives = 10
batch_size = 256
verbose = 1

# Training model
for epoch in xrange(num_epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)
    
    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                      np.array(labels), # labels 
                      batch_size=batch_size, epochs=1, verbose=0, shuffle=True)
    t2 = time()
    
    # Evaluation
    if epoch % verbose == 0:
        (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]' 
              % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch

print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))

Iteration 0 [266.9 s]: HR = 0.1315, NDCG = 0.0673, loss = 1.4023 [121.7 s]
Iteration 1 [251.8 s]: HR = 0.1315, NDCG = 0.0673, loss = 1.4023 [118.7 s]
Iteration 2 [249.4 s]: HR = 0.1315, NDCG = 0.0673, loss = 1.4023 [127.3 s]
End. Best Iteration -1:  HR = 0.1315, NDCG = 0.0673. 


## Полезные ссылки:

### Пейпер, на который все ссылаются:
Xiangnan He, Lizi Liao, Hanwang Zhang, Liqiang Nie, Xia Hu & Tat-Seng Chua, Neural Collaborative Filtering, 2017 https://arxiv.org/abs/1708.05031

### Туториалы и реализации:
##### Microsoft:
https://github.com/microsoft/recommenders/blob/master/notebooks/02_model/ncf_deep_dive.ipynb

##### Tensorflow:
https://github.com/tensorflow/models/tree/master/official/recommendation

##### Towards Data Science
https://towardsdatascience.com/neural-collaborative-filtering-96cef1009401

### Лекции:
Е.Соколов. Рекомендательные системы. Лекция 1 https://github.com/hse-ds/iad-applied-ds/blob/master/2020/lectures/lecture01-recommender.pdf

Е.Соколов. Рекомендательные системы. Лекция 2 https://github.com/hse-ds/iad-applied-ds/blob/master/2020/lectures/lecture02-recommender.pdf

